# Entregable 1. 
Técnicas de Recogida de Datos. <br>
MBDS Universidad de Navarra 2023/2024 <br>
Paula Sanjuan Campos

## Web scraping exercise

Define a generic function `SOS_help` which retrieves help results from Stack Overflow Stunning results. <br>


### print_out function

`print_out` prints command execution

In [1]:
def print_output(command):
    command = 'print(' + command + ')'
    return exec(command)

In [2]:
import numpy as np

In [3]:
print_output(command = 'np.random.uniform(-1, 1, size=100)')

[-0.45993175  0.63994246 -0.48930471  0.98202975  0.34658855 -0.1750249
 -0.41701006  0.6979908   0.64748952 -0.63055864 -0.84399103  0.03194298
  0.70873479 -0.20249189 -0.39262424 -0.12354557 -0.17053768  0.46522177
  0.45626354  0.80380336  0.43332603  0.69375972  0.46166476 -0.88889523
  0.81990013  0.56967641 -0.06131613 -0.5590022   0.53531737 -0.40777723
 -0.58584808  0.05511568 -0.30967205 -0.47836725 -0.57757834  0.40940735
 -0.24842466  0.73196494 -0.59591411 -0.70941815 -0.45720926 -0.41194129
  0.60416516  0.91313938 -0.46948641 -0.39445152 -0.49988762  0.30635201
  0.6161018  -0.04739583  0.15357575  0.84305259  0.17522432  0.64039805
  0.65388557  0.36671115 -0.81978712  0.37643988  0.02890947  0.18733375
 -0.53964945  0.71823084  0.12801088  0.52094863 -0.06134235 -0.13178848
 -0.04324635 -0.52990902 -0.35774114  0.63957179  0.65517149 -0.47931442
 -0.29097385 -0.03281655  0.4594047   0.68884511 -0.44746389  0.11329554
  0.75701109  0.00797963 -0.23547752 -0.81975656 -0.

if `print_out` results in an error, this is printed

In [4]:
print_output(command = 'nn.random.uniform(-1, 1, size=100)')

NameError: name 'nn' is not defined

In [ ]:
print_output(command = 'np.random.uniform(-1, 1, siz=100)')

TypeError: uniform() got an unexpected keyword argument 'siz'

### get_SOS_help function

Create a function `get_SOS_help` which: <br>
    - Prints "works as intended" if no error. <br>
    - Prints the first link from stack overflow related to the error. As an example: <br>
        `print_output(command = 'np.random.uniform(-1, 1, siz=100)'`
        should retrieve the following link:
        https://stackoverflow.com/questions/72537485/typeerror-uniform-got-an-unexpected-keyword-argument-low-size <br>
    - Prints the most voted help
    - Opens a new browser using the link

In [6]:
!pip install selenium


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# MODULES AND LIBRARIES
import time
import os
import time
import requests
import pandas as pd

from lxml import html
import numpy as np

from bs4 import BeautifulStoneSoup # Extract data from HTML and XML files. Analyze the structure of a web page and extract information.
from selenium import webdriver # control and automation of actions in web browsers
from urllib.parse import quote # convert text to URL format
import requests # send HTTP requests in Python (interaction with web sites)
from bs4 import BeautifulSoup
import re # manipulate regular expressions

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    WebDriverException,
    )
from selenium.webdriver.support.ui import Select

Next, the necessary configuration to control a Chrome browser is set up and a `webdriver.Chrome` object is created, which represents an instance of the browser controlled with Selenium. This window is needed to get the content of the google search (first part of the exercise) and to display the content of the Stack Overflow link and the solutions sorted by score (second part of the exercise). So, it is important not to close this navigation window while the notebook is running. If it is closed, the previous cell must be run again.

In [10]:
# control and automation of actions in web browsers
from selenium import webdriver # webdriver module of the Selenium library
from selenium.webdriver.chrome.service import Service # control the Chrome browser process
from webdriver_manager.chrome import ChromeDriverManager # automatically manage Chrome driver downloads and updates
from selenium.webdriver.chrome.options import Options # configure browser-specific options

chrome_options = Options()

# Add any specific options you need here, for example:
# chrome_options.add_argument('--headless')

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

#### Secondary functions



```
print_error:
```

Function that prints the error and the type of error being made when executing the command. Returns the error URL format.


In [11]:
def print_error(command):
    try:
        exec(command)
        print('Works as intended')
        return None
    except Exception as e:

        # Error format
        e_type = type(e).__name__
        e_message = str(e)
        formatted_e = quote(e_type + ": " + e_message) 
        print(e_type + ": " + e_message)
        return formatted_e;




```
get_stackoverflow_url:
```

Function that in case of finding a Stack Overflow link in the  google browser returns it. Otherwise, it prints on the screen that no Stack Overflow link was found.

1. Generates the URL to search in google by joining the google search header with the error returned by `print_error` (variable received by the function).
2. With the generated url and with webdriver it obtains the content of the browser that is stored in the `html` variable. 
3. With BeautifulSoup it looks for the appropriate tags to get the links of the different options of the `html` (`hrefs`) until it finds the first one that matches the beginning of the syntax of the Stack Overflow links ('http://stackoverflow). This has been done with the `re` module that provides functions to work with regular expressions.
4. If it finds a Stack Overflow link it returns it, otherwise it prints 'No stackoverflow links found'.




In [12]:
def get_stackoverflow_url(search_query):

    # Send request to Google
    google_url = f"https://www.google.com/search?q={search_query}"
    # response = requests.get(google_url)
    # print('google link: ', google_url)

    WAIT_TIME = 10
    driver.wait = WebDriverWait(driver, WAIT_TIME)

    # Opening the browser and saving the content
    driver.get(google_url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Get Stack Overflow link
    search_titles = soup.find_all('a', attrs={'data-ved': True}, href=True)
    hrefs = [re.search(r'href="(.*?)"', str(title)).group(1) for title in search_titles]
    stackoverflow_link = next((href for href in hrefs if re.search(r'^(http|https)://(www\.)?stackoverflow\.com', href)), None)
    if stackoverflow_link:
        print('First stackoverflow link: ', stackoverflow_link)
        return stackoverflow_link
    else:
        print('No stackoverflow links found.')

```
get_most_voted_answer:
```

Function that receives the Stack Overflow URL. It opens the received URL in a browser window, which must be the Stack Overflow page corresponding to the error that has occurred. Then it sorts the answers ('answers' section) by score ('Highest score'). Although this is the default, it is configured by selecting index 0 from the drop-down menu. Finally, the solution with the highest score is printed.

In [16]:
def most_voted_answer(stackoverflow_link):

    # Open the StackOverflow URL obtained previously
    WAIT_TIME = 10
    driver.wait = WebDriverWait(driver, WAIT_TIME)
    driver.get(stackoverflow_link)

    # Identify cookies button and click it if it is necessary
    try:
        cookies_message = driver.find_element(By.ID, "onetrust-banner-sdk")
        accept_cookies_button = cookies_message.find_element(By.XPATH, './/button[text()="Accept all cookies"]')
        accept_cookies_button.click()
    except NoSuchElementException:
        print("Cookies message element not found, already dismissed.")
    
    # Select the default option in the dropdown menu (index = 0 in Stack Overflow)
    sorting_dropdown_element = driver.find_element(By.ID, "answer-sort-dropdown-select-menu")
    sorting_dropdown_element_select = Select(sorting_dropdown_element)
    sorting_dropdown_element_select.select_by_index(0) 

    # Print the first and most voted answer
    first_answer = driver.find_element(By.CSS_SELECTOR, ".answer.js-answer")
    answer_body = first_answer.find_element(By.CSS_SELECTOR, ".s-prose.js-post-body")
    print(answer_body.text)




```
get_SOS_help
```
Print the error and the Stack Overflow link. Then, in the browser, the link is opened and the help options to print the content of the top rated help are configured.


In [14]:
def get_SOS_help(command):
    try:
        exec(command)
        print('Works as intended')
    except Exception as e:

        # Get the error
        formatted_e = print_error(command) 

        # Get and print the Stack Overflow URL
        search_query = formatted_e
        stackoverflow_link=get_stackoverflow_url(search_query)

        # Configure the web and print the most voted solution
        # print('Most voted answer: \n')
        most_voted_answer(stackoverflow_link)


In [17]:
command = 'np.random.uniform(-1, 1, siz=100)'
get_SOS_help(command)

TypeError: uniform() got an unexpected keyword argument 'siz'
First stackoverflow link:  https://stackoverflow.com/questions/72537485/typeerror-uniform-got-an-unexpected-keyword-argument-low-size
You must declare the alias library or the library name directly
        import numpy as np
        
        def apply_net(y_in):
          global w, b
        
          z=np.dot(w, y_in)+b
          return(1/(1+np.exp(-z)))
        
        N0=2
        N1=1 
        
        w=np.random.uniform(low=-10,high=+10,size=(N1,N0)) # random weights: N1xN0
        b=np.random.uniform(low=-1,high=+1,size=N1) #biases: N1 vector


In [18]:
driver.quit()